In [3]:
import pandas as pd
import sqlite3

In [4]:
connection = sqlite3.connect("../data/checking-logs.sqlite")

In [5]:
pd.io.sql.read_sql("PRAGMA table_info(test);", connection)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [6]:
pd.io.sql.read_sql("SELECT * FROM test LIMIT 10;", connection)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [7]:
pd.io.sql.read_sql("PRAGMA table_info(deadlines);", connection)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [8]:
pd.io.sql.read_sql("SELECT * FROM deadlines LIMIT 10;", connection)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [15]:
query = """
SELECT 
    uid,
    (deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600 AS min_value
FROM 
    test as T INNER JOIN deadlines as D ON T.labname = D.labs
WHERE 
    labname != 'project1'
GROUP BY
    uid, labname
ORDER BY 
    min_value
LIMIT 1
"""

df_min = pd.io.sql.read_sql(query, connection)
df_min

,uid,min_value
0,user_25,2


In [16]:
query = """
        SELECT 
            uid,
            (deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600 AS max_value
        FROM 
            test as T INNER JOIN deadlines as D ON T.labname = D.labs
        WHERE 
            labname != 'project1'
        GROUP BY
            uid, labname
        ORDER BY 
            max_value DESC
        LIMIT 1
        """
df_max = pd.io.sql.read_sql(query, connection)
df_max

,uid,max_value
0,user_30,202


In [17]:
query = """
        SELECT 
            AVG((deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600) AS avg_value
        FROM 
            test as T INNER JOIN deadlines as D ON T.labname = D.labs
        WHERE 
            labname != 'project1'
        """
df_avg = pd.io.sql.read_sql(query, connection)
df_avg

,avg_value
0,89.125


In [18]:
query = """
        SELECT 
            uid,
            AVG((deadlines - strftime('%s', DATETIME(first_commit_ts))) / 3600) AS avg_diff,
            COUNT(first_view_ts) as pageview
        FROM 
            test as T INNER JOIN deadlines as D ON T.labname = D.labs
        WHERE 
            labname != 'project1'
        GROUP BY
            uid
        """
views_diff = pd.io.sql.read_sql(query, connection)
views_diff

,uid,avg_diff,pageview
0,user_1,64.400000,5
1,user_10,74.800000,5
2,user_14,159.000000,3
3,user_17,61.600000,5
4,user_18,5.666667,3
5,user_19,98.750000,4
6,user_21,95.500000,4
7,user_25,92.600000,5
8,user_28,86.400000,5
9,user_3,105.400000,5


In [19]:
correlation = views_diff['pageview'].corr(views_diff['avg_diff'])
correlation

np.float64(-0.11768489475608129)

In [21]:
connection.close()